In [58]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [59]:
df = pd.read_csv(dirname + '/train.csv')
sub = pd.read_csv(dirname + '/sample_submission.csv')

In [60]:
RANDOM_SEED = 42 

Посмотрим данные ближе

# USER_ID

In [61]:
df['user_id'].nunique()

In [62]:
v_c_user = df['user_id'].value_counts()

In [63]:
idxs_users_upper_mean = set(v_c_user[v_c_user > v_c_user.mean()].index) # Индексы пользователей, которые,в среднем, делают больше заказов.

In [64]:
df['user_id'].value_counts().hist(bins=150, figsize=(20,6));

Видим, что имеем 20000 уникальных пользователей. Постоянные клиенты (с меньшим id) делают больше заказов. Можем предположить, что они уже разобрались со своими предпочтениями, и скорее всего, закажут то, что ранее заказывали. 

# CART

In [65]:
plt.figure(figsize=(20,7))
plt.hist(df.cart, bins=300)
plt.title('Рспределение всех позиций');

Мы можем видеть, что некоторые позиции выбираются гораздо чаще остальных среди всей выборки пользователей. Возможно это продукты постоянного потребления. Так же можно заметить некоторую кластеризацию позиций, например с 380 по 540. Возможно используем KMeans.

Предположим, что вероятность того, закажет ли пользователь определенную позицию связана с частотой заказов этой позиции по всему датасету. Чем чаще встречается, тем больше вероятность заказа.

Для эксперемента возьмем индексы позиций частота которых выше  средней, медианной и прозвольных (исходя из графика) частот. 

In [66]:
v = df.cart.value_counts()
print('Медианная частота позиции {}'.format(v.median()))
print(f'Средняя частота позиции {round(v.mean(),1)}')

In [67]:
carts_upper_mean_v = set(map(str,v[v.values>=np.mean(v.values)].index))
carts_upper_median_v = set(map(str,v[v.values>=np.median(v.values)].index))
# carts_upper_75_v = set(map(str,v[v.values>=75000].index))
# carts_upper_90_v = set(map(str,v[v.values>=90000].index))
# carts_upper_100_v = set(map(str,v[v.values>=100000].index))
# carts_upper_50_v = set(map(str,v[v.values>=50000].index))

Сгрупируем датасет по user_id и cart, посчитаем количество заказов каждой позиции

In [68]:
grouped_df = pd.DataFrame(df.groupby(['user_id', 'cart'])['order_completed_at'].count())
grouped_df.columns=['amount_of_orders']

In [69]:
grouped_df.sample(5)

In [70]:
plt.figure(figsize=(20,6))
plt.plot(grouped_df.groupby(level='cart').count())
plt.title('Распределение позиций после группировки');

Мы видим, что графики очень похожи. Проверим совпадают ли множества индексов с частотами позиций больше медианной и средней частот.

In [71]:
g = grouped_df.groupby(level='cart').count()
carts_upper_mean_g = set(map(str,g[g['amount_of_orders']>=g['amount_of_orders'].mean()].index)) 
carts_upper_median_g = set(map(str,g[g['amount_of_orders']>=g['amount_of_orders'].median()].index)) 

In [72]:
carts_upper_mean_g - carts_upper_mean_v

In [73]:
carts_upper_median_g - carts_upper_median_v

Видим, что множества имеют различия. Может имеет смысл их объеденить?

In [74]:
carts_upper_median_v.update(carts_upper_median_g)

In [75]:
carts_upper_mean_v.update(carts_upper_mean_g)

In [76]:
indxs = [str(x[0])+';'+str(x[1]) for x in grouped_df.index] # Преобразуем индексы к виду, который требует задание.
count_of_orders = grouped_df.values

In [77]:
grouped_df

In [78]:
df_new = pd.DataFrame()
df_new['id'] = indxs
df_new['count_of_orders'] = count_of_orders

In [79]:
df_new.count_of_orders.value_counts()[:15]

Можем отметить, что продуктов, которые купили 1,2 или 3 раза больше всего. Предположим, вероятность того, что приобретут товары вне этого диапазона.

In [80]:
ids_2 = set(df_new[(df_new['count_of_orders']>=3)]['id'].reset_index(drop=True)) # Отберем индексы вида user_id;cart выполняющие условия

In [105]:
sub['target'] = sub['id'].apply(lambda x:1 if x in ids_2 and 
                                ((x.split(';')[1] in carts_upper_mean_v) or 
                                (x.split(';')[0] in idxs_users_upper_mean)) else 0 )

In [106]:
sub['target'].value_counts(normalize=True)

In [83]:
sub.to_csv('base_sub.csv', index=False)

Такое решение выдало 0.41347. Попробуем добавить кластеры и проанализировать их.

In [84]:
pd.read_csv('/kaggle/working/base_sub.csv')

In [85]:
from sklearn.cluster import KMeans

Разобьем продукты на 6 кластеров по частоте их заказов. (Интуитивно, исходя из графика)

In [86]:
X = df['cart'].reset_index(drop=True).values
X = X.reshape(-1,1)

In [87]:
model_cart = KMeans(n_clusters=6, random_state=RANDOM_SEED)
predict = model_cart.fit_predict(X)


In [88]:
df['cluster_cart'] = predict

In [89]:
df.cluster_cart.hist(bins=12);

Попробуем кластеризовать, добавив к информации о продуктах информацию о пользователях

In [90]:
XXX = df[['user_id', 'cart']].values

In [91]:
model__ = KMeans(n_clusters=6, random_state=RANDOM_SEED)
predict__ = model__.fit_predict(XXX)

In [92]:
df['cluster_user_cart'] = predict__

In [93]:
df['cluster_cart'].plot(kind='hist', bins=12);

In [94]:
df['cluster_user_cart'].plot(kind='hist', bins=12)

Предположим, что записи с кластерами, которые встречаются чаще с большей вероятностью купят еще. А именно (0, 3, 5) в двух новых столбцах

In [95]:
zzz = df.query('(cluster_user_cart in [1, 3, 5]) and cluster_cart in [0, 3, 5]') # Запрашиваем данные, которые выполняют условия выше

In [96]:
def make_idx(user_id, cart):
    return str(user_id) + ';' + str(cart)

In [97]:
idxs_in_both_clusters = set(zzz[['user_id', 'cart']].apply(lambda x:make_idx(*x), axis=1))

In [98]:
sub.target.hist()

In [99]:
sub.to_csv('base_sub.csv', index=False)

In [100]:
pd.read_csv('/kaggle/working/base_sub.csv')

Способ с кластеризацией показал себя хуже. Возможно из-за неверно выбранного количества кластеров или ошибочного интерпретирования их влияния.